In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import json
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split

os.environ['CUDA_VISIBLE_DEVICES']='2'
pd.set_option('display.max_colwidth', None)

sys.path.append('../../lib/')
sys.path.append('../../lib/counter_gen_lib')

In [3]:
from utility import *
import interact

In [22]:
import torch

In [21]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


### Data Preparation:

In [7]:
data_path = '/workspace/ceph_data/argument-undermining/'

In [13]:
jo_df =  pd.read_pickle(data_path + "jo_data/gen_data.pickle")

training_df = jo_df[jo_df.split =='train']
valid_df = jo_df[jo_df.split =='val']

In [ ]:
training_df = utility.clean_df(training_df)
valid_df    = utility.clean_df(valid_df)

In [ ]:
jo_training = utility.prepare_data_for_training({'train': training_df, 'valid': valid_df[0:5000]},
                                                full_counter=False, context='title+post', 
                                                post_clm='post', 
                                                comment_clm='comment_sents', 
                                                attacks_clm='premise_counter_premise_pairs', max_sens=20, baseline=False)

In [ ]:
json.dump(jo_training, open(data_path + 'gpt_data/jo_data.json', 'w'))

### Training GPT:

In [17]:
GENERATION_LIB_PATH='/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib'

### Training the model with weak premises annotated:

In [ ]:
!CUDA_VISIBLE_DEVICES=6 $sys.executable /workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json \
                    --model_checkpoint gpt2 \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --output_model_checkpoint my_approach_v2_4_epochs \
                    --dataset_cache ./jo_data_cache \
                    --build_instance_version 'v2' \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 4

INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Arguments: Namespace(baseline=False, build_instance_version='v2', dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='gpt2', n_epochs=4, num_candidates=2, output_model_checkpoint='my_approach_v2_4_epochs', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Prepare tokenizer, pretrained model and optimizer.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/to

### Training the model with weak premises annotated, and with special tokens:

In [ ]:
!CUDA_VISIBLE_DEVICES=6 $sys.executable /workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json \
                    --model_checkpoint gpt2 \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --output_model_checkpoint my_approach_v3_4_epochs \
                    --dataset_cache ./jo_data_cache \
                    --build_instance_version 'v3' \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 4

INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Arguments: Namespace(baseline=False, build_instance_version='v3', dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='gpt2', n_epochs=4, num_candidates=2, output_model_checkpoint='my_approach_v3_4_epochs', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Prepare tokenizer, pretrained model and optimizer.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/to

### Training the baseline:

In [ ]:
!CUDA_VISIBLE_DEVICES=6 $sys.executable /workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py \
                    --dataset_path /workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json \
                    --model_checkpoint gpt2 \
                    --log_dir /workspace/ceph_data/argument-undermining/models/gen_models/ \
                    --baseline \
                    --dataset_cache ./jo_data_cache \
                    --output_model_checkpoint jo_baseline_gpt2 \
                    --num_candidates 2 \
                    --train_batch_size 1 \
                    --valid_batch_size 1 \
                    --lr 6.25e-5 \
                    --n_epochs 4

INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Arguments: Namespace(baseline=True, build_instance_version=None, dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='gpt2', n_epochs=4, num_candidates=2, output_model_checkpoint='jo_baseline_gpt2', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/counter-argument-generation-via-undermining/code/lib/counter_gen_lib/train.py:Prepare tokenizer, pretrained model and optimizer.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /root/.cache/torch/tran